In [1]:
print('Hello, World!')

StatementMeta(spark24pool, 0, 1, Finished, Available)

Hello, World!

In [7]:
container_name = 'synstorage'
storage_account_name = 'strdbwsynworkshop'
df = spark.read.format('delta').load(f'abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/delta_test')

StatementMeta(spark24pool, 0, 7, Finished, Available)

In [8]:
display(df)

StatementMeta(spark24pool, 0, 8, Finished, Available)

SynapseWidget(Synapse.DataFrame, d1e4dd59-45dd-41e2-ad74-522d8f318adc)